In [2]:
import os

In [3]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/saikiven03/DS_Project1.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='saikiven03'
os.environ['MLFLOW_TRACKING_PASSWORD']='7c0e76b0f174dd060d5f14dafb6d79494ef83cfa'

In [4]:
import os
%pwd

'd:\\MLOPS_udemy_Python\\DS_Project1\\research'

In [5]:
os.chdir('../')
%pwd

'd:\\MLOPS_udemy_Python\\DS_Project1'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [8]:
from src.DataScience_Proj1.constants import *
from src.DataScience_Proj1.utils.common import read_yaml, create_directories,save_json

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_PATH_FILE ,
                 params_filepath = PARAMS_PATH_FILE,
                 schema_filepath = SCHEMA_PATH_FILE):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_eval_cofig(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.Elasticnet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri= 'https://dagshub.com/saikiven03/DS_Project1.mlflow'
        )
        
        return model_evaluation_config


In [11]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [12]:
##Model evaluation
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import mlflow
import pandas as pd
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        mse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return mse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y= test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        ###Starting mlflow run
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (mse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            #saving metrics as local
            scores = {"mse": mse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("mse", mse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            #model registry
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [15]:
try:
    config=ConfigurationManager()
    model_eval_config=config.get_model_eval_cofig()
    model_eval_config=ModelEvaluation(config=model_eval_config)
    model_eval_config.log_into_mlflow()
except Exception as e:
    raise e


[2025-01-16 17:37:18,982: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-16 17:37:18,988: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-16 17:37:18,994: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-16 17:37:18,997: INFO: common: created directory at: artifacts]
[2025-01-16 17:37:19,000: INFO: common: created directory at: artifacts/model_evaluation]
[2025-01-16 17:37:21,395: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/01/16 17:37:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2025/01/16 17:37:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run treasured-stork-769 at: https://dagshub.com/saikiven03/DS_Project1.mlflow/#/experiments/0/runs/1d6382eb93244461b3fa6c49239f48cb
🧪 View experiment at: https://dagshub.com/saikiven03/DS_Project1.mlflow/#/experiments/0
